<a href="https://colab.research.google.com/github/EAakiyama3104/python_lecture/blob/master/%5BPython%E8%AC%9B%E5%BA%A7%5D%E7%AC%AC5%E5%9B%9E%E6%AD%8C%E8%A9%9E%E3%82%B5%E3%82%A4%E3%83%88%E3%81%AE%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Webスクレイピングとは


* Webサイトから情報を取得する技術
* APIを公開していないサイトの情報も取得できる
* HTMLを読み込んで、必要な情報を抜き出す必要がある

手順


1. HTTPリクエストを送り、HTMLのレスポンスを取得
2. HTMLの中身を見て、必要な情報がある場所を確認
3. HTMLから必要な情報を抽出





## アーティストのページから歌詞ページ一覧を取得

今回は、Uta-Net からあるアーティスト(米津玄師)の歌詞をすべて取得。

以下のページから情報を取得

* アーティストページ - 曲ページのURLをすべて取得
* 曲ページ(歌詞ページ) - 歌詞を取得

requests について

* PythonのコードからHTTPリクエストを送るためのライブラリ
* 使い方が簡単


https://requests-docs-ja.readthedocs.io/en/latest/

beatiful soup について

* HTMLやXMLからデータを抽出するためのライブラリ
* 様々な条件でHTML上のデータの位置を指定

機能が多いので、使用の際はリファレンスを読むことをおすすめします。
http://kondou.com/BS4/

In [ ]:
# 必要なライブラリをインポート
import bs4
import requests
import time
from pprint import pprint

In [ ]:
# アーティストページ(米津玄師)のアドレス
ARTIST_URL = 'https://www.uta-net.com/artist/11591/'

# 曲ページの先頭アドレス
BASE_URL = 'https://www.uta-net.com'

In [ ]:
# ページを読み込むFunction
def load_page(url):
  response = requests.get(url)
  response.raise_for_status()
  return response.text

In [ ]:
artist_page = load_page(ARTIST_URL)

In [ ]:
artist_page

In [ ]:
# タグをすべて取得する関数
def get_tags(page, target_tag):
  soup = bs4.BeautifulSoup(str(page), 'html.parser')
  tags = soup.find_all(target_tag)
  return tags

In [ ]:
# td タグをすべて取得
td_list = get_tags(artist_page, 'td')
td_list

In [ ]:
# 最初のtdタグを取得
td_list[0]

In [ ]:
# 最初のtdタグからaタグを取得
a_list = get_tags(td_list[0], 'a')
a_list

In [ ]:
# href属性を見て、URLに'song'が含まれているかを確認
a_list[0].get('href')

In [ ]:
if 'song' in a_list[0].get('href'):
  print(BASE_URL + a_list[0].get('href'))

In [ ]:
# 歌詞ページのURLのリスト
track_urls = []

In [ ]:
# すべての曲のURLを取得
# td タグのリストを1つずつ処理
for td in td_list:
  # a タグのリストを取得し、1つずつ処理
  for a in get_tags(td, 'a'):
    # aタグのhref属性を取得
    href = a.get('href')
    # URLに'song'が入っている場合
    if 'song' in href:
      # track_urls に完全なURLを追加
      track_urls.append(BASE_URL + href)

In [ ]:
# URLのリスト
track_urls

## 歌詞ページから歌詞を取得

歌詞ページの例: https://www.uta-net.com/song/162135/

In [ ]:
track_urls[0]

In [ ]:
lyric_page = load_page(track_urls[0])

In [ ]:
lyric_page

In [ ]:
soup = bs4.BeautifulSoup(lyric_page, 'html.parser')

In [ ]:
# id が kashi_area のタグを探す
soup.find(id='kashi_area')

In [ ]:
# テキストのみを抽出
soup.find(id='kashi_area').text

In [ ]:
# 全角スペースが文字化けしているので、半角スペースに置換
soup.find(id='kashi_area').text.replace('　', '\n')

In [ ]:
soup.find(id='kashi_area').text.split('')

In [ ]:
for sentence in soup.find(id='kashi_area').text.split('　'):
  print(sentence)

In [ ]:
# 歌詞を入れるリスト
lyrics = []

In [ ]:
# すべての曲の歌詞をlyricsに入れる
for track_url in track_urls:
  lyric_page = load_page(track_url)
  soup = bs4.BeautifulSoup(lyric_page, 'html.parser')
  lyrics.append(soup.find(id='kashi_area').text.replace('　', ' '))
  time.sleep(1)

In [ ]:
lyrics[:10]

In [ ]:
len(lyrics)

## MeCab を用いて形態素解析

MeCabのインストール

In [103]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?
/bin/sh: 1: aptitude: not found


In [104]:
# 最新の辞書をダウンロード
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab is not found.


# MeCabで1曲ずつ形態素解析を行う

正直いらないかも。。。

形態素解析とは

文章を意味をもつ最小単位の表現に分解すること。最小単位は単語になることが多い。

In [106]:
# MeCab セットアップ
import MeCab
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
mecab = MeCab.Tagger("-d {0}".format(path))
mecab.parse('')

RuntimeError: 

In [109]:
mecab_parser=lambda x: MeCab.Tagger("-Owakati").parse(x).split(" ")

In [110]:
mecab_parser('選挙の日ってうちじゃなぜか投票行って外食するんだ')

['選挙',
 'の',
 '日',
 'って',
 'うち',
 'じゃ',
 'なぜ',
 'か',
 '投票',
 '行っ',
 'て',
 '外食',
 'する',
 'ん',
 'だ',
 '\n']

In [111]:
for word in mecab.parse('選挙の日ってうちじゃなぜか投票行って外食するんだ').split('\n'):
  print(word)

AttributeError: 'function' object has no attribute 'parse'

In [ ]:
mecab.parse('おのののか')

In [ ]:
mecab.parse(lyrics[0])

In [ ]:
node = mecab.parseToNode(lyrics[0])

In [ ]:
while node:
  #単語を取得
  word = node.surface
  #品詞を取得
  pos = node.feature.split(',')[0]
  if pos in ['名詞','形容詞','副詞','動詞']:
    print(word)
  #次の単語に進める
  node = node.next

In [ ]:
# すべての曲の単語を入れる配列
words = []

for lyric in lyrics:
  node = mecab.parseToNode(lyric)
  while node:
    #単語を取得
    word = node.surface
    #品詞を取得
    pos = node.feature.split(',')[0]
    if pos in ['名詞','形容詞','副詞','動詞']:
      words.append(word)
    #次の単語に進める
    node = node.next

In [ ]:
len(words)

collections.Counter を使って単語の出現回数をカウント



In [ ]:
import collections

word_counter = collections.Counter(words)

In [ ]:
# 何種類の単語があるか
len(word_counter)

In [ ]:
# 単語の出現回数
word_counter['あなた']

In [ ]:
# 出現回数が多い順に並べる
word_counter.most_common()

## ワードクラウドで可視化する

ワードクラウド（word cloud）とは単語を出現回数に比例する大きさで雲のように並べたもの


例

![代替テキスト](https://oku.edu.mie-u.ac.jp/~okumura/python/img/wc1.png)


Python では wordcloud モジュールで作成

In [ ]:
# 日本語フォントをインストール
!apt-get -y install fonts-ipafont-gothic

In [ ]:
import matplotlib.pyplot as plt
import wordcloud

In [ ]:
# 日本語フォントのパス
fpath = '/usr/share/fonts/truetype/fonts-japanese-gothic.ttf'

In [ ]:
# wordcloud で使うために、単語のリストをスペース区切りの文字列に変換
words_str = ' '.join(words)

In [ ]:
lyric_cloud = wordcloud.WordCloud(background_color="white", 
                                  font_path=fpath, 
                                  width=900, 
                                  height=500).generate(words_str)

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(lyric_cloud)
plt.axis("off")
plt.show()

意味を持たない単語が多くなってしまう

これを防ぐのが、stopwords。stopwordsで与えられた単語をword cloud から除外する

In [ ]:
stop_words = [ 'てる', 'いる', 'なる', 'れる', 'する', 'ある', 'こと', 'これ', 'さん', 'して', \
             'くれる', 'やる', 'くださる', 'そう', 'せる', 'した',  '思う',  \
             'それ', 'ここ', 'ちゃん', 'くん', '', 'て','に','を','は','の', 'が', 'と', 'た', 'し', 'で', \
             'ない', 'も', 'な', 'い', 'か', 'ので', 'よう', '']

In [ ]:
lyric_cloud = wordcloud.WordCloud(background_color="white", 
                                  font_path=fpath,
                                  width=900, 
                                  height=500,
                                  stopwords=stop_words).generate(words_str)

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(lyric_cloud)
plt.axis("off")
plt.show()

# BERTで歌詞をしてみる（秋元康編）


In [6]:
!pip install tqdm
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [7]:
# Bertのコンポーネントの transformers をダウンロード
!pip install transformers
!git clone https://github.com/loverico/transformers.git
!mkdir output


Cloning into 'transformers'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 20079 (delta 0), reused 1 (delta 0), pack-reused 20075
Receiving objects: 100% (20079/20079), 11.63 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (14673/14673), done.


## BERT とは
自然言語処理に特化した事前学習済み機械学習モデル。
人間の文章認識能力を超えたことが話題になった。
### 事前学習済みモデルとは
あらかじめ汎化性のあるタスクを学習しておいたモデルのこと。
例えば、BERT の場合は、欠損した文章を与えてその欠損した箇所にはどんな単語が入るか推測させるなどのタスクを行う
その学習されたモデルを用いて、個別のタスクで更に学習させることによって効果的な学習結果が期待できる(→転移学習と呼ぶ）



In [93]:
# url
import urllib.parse
from tqdm import tqdm_notebook as tqdm
#人気順（昇順）
POPULAR_ASC = 3
# 人気順（降順)
POPULAR_DESC = 4
# 作詞者氏名で検索
SEARCH_TYPE_WRITER=3

class LyricFetcher():
    artist_name = None
    base_url = None
    lyrics_list = []
    stop_words = []
    def __init__(self,artist_name,base_url=BASE_URL):
        self.artist_name = artist_name

    def build_list_url(self, keyword, search_type=SEARCH_TYPE_WRITER, sort_type=POPULAR_ASC):
        return f"https://www.uta-net.com/search/?Aselect={search_type}&Bselect=3&Keyword={urllib.parse.quote(keyword)}&sort={sort_type}"
    
    def remove_stop_words(self,words=[], stop_words=[]):
        if len(stop_words)> 0:
            self.stop_wrods = stop_words
        return [word for word in words if word not in self.stop_words]
        
    def extract_track_urls(self, td_list, fetch_cnt):
        track_urls = []
        for td in td_list:
          # a タグのリストを取得し、1つずつ処理
          for a in self.get_tags(td, 'a'):
            # aタグのhref属性を取得
            href = a.get('href')
            # URLに'song'が入っている場合
            if 'song' in href:
              # track_urls に完全なURLを追加
              track_urls.append(BASE_URL + href)
        print(len(track_urls[:fetch_cnt]))
        return track_urls[:fetch_cnt]

    def get_tags(self, page, target_tag):
        soup = bs4.BeautifulSoup(str(page), 'html.parser')
        tags = soup.find_all(target_tag)
        return tags

    def load_page(self, url):
        response = requests.get(url)
        response.raise_for_status()
        return response.text 

    def fetch_lyric(self, track_url):
        if track_url == "":
            return None
        lyric_page = load_page(track_url)
        soup = bs4.BeautifulSoup(lyric_page, 'html.parser')
        # アクセス過多を防ぐために１秒間休止
        time.sleep(1)
        return soup.find(id='kashi_area').text.replace('　', ' ')

    def run_fetch(self, fetch_cnt=10, sort_type=POPULAR_ASC):
        # url 作成
        url= self.build_list_url(self.artist_name, sort_type)
        # アーティストページ取得
        artist_page= self.load_page(url)
        # td_list を作成
        td_list = self.get_tags(artist_page, 'td')
        # そこから曲のurlリストを抽出
        track_urls= self.extract_track_urls(td_list, fetch_cnt)

        for track_url in tqdm(track_urls):
            self.lyrics_list.append(self.fetch_lyric(track_url))
    
    # return self.lyrics_list

In [94]:
akimoto_lyrics = LyricFetcher("秋元康")
akimoto_lyrics.run_fetch(fetch_cnt=20, sort_type=POPULAR_ASC)

20


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [16]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
track_token=tokenizer.tokenize(akimoto_lyrics.lyrics_list[0])

In [12]:
akimoto_lyrics.lyrics_list[0]

'君も頑張れ そうだ みんなして頑張ろうしあわせは誰かからもらうものじゃなく掴むものその手を伸ばせよ今 この世界には見えない悲しみがどれだけ存在して 人々を泣かすのか？地球上のどこかに涙を溜めてる貯水池とかがあるなら 底の栓を抜いてしまえ一日が終わるまでに日向(ひなた)を探そう暖かくてほっとするそう 自分の居場所がきっと見つかるだろう僕も頑張る だから みんなも頑張れ誰も諦めなければ 未来だって輝いてるよ夢を見続けよう もしもそれが叶わなくても何かゴールがあったらモチベーションだ一緒に笑顔になろうなぜ笑ってる人と泣いてる人がいるの？しあわせの分配は 元々 同じなのに…誰かが何かを独り占めしたのか？それとも誰かに何かをただ押し付けられたのか？眠れない長い夜は夢でも見るんだわくわくして楽しいよああ 他人と比較をしても意味がない空を頑張る 天気 まだまだ崩れるな雨 落ちて来なければ 夕陽だって美しいだろう夜は待っているか？ やがて星は仲間を連れて暗い時間に希望を生(う)もうとするすべてを全力で行こう僕も頑張る だから みんなも頑張れ誰も諦めなければ 未来だって輝いてるよ夢を見続けよう もしもそれが叶わなくても何かゴールがあったらモチベーションだ一緒に笑顔になろう'

In [97]:
from transformers.optimization import AdamW, WarmupLinearSchedule

ImportError: cannot import name 'WarmupLinearSchedule' from 'transformers.optimization' (/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py)

In [96]:
from transformers import get_linear_schedule_with_warmup

In [113]:
#file　書き込み
path_w = "./hoge.txt"
with open(path_w, mode='w') as f:
    
    for lyric in akimoto_lyrics.lyrics_list:
        row= mecab_parser(lyric)
        f.write(" ".join(row))    